In [35]:

import os
import glob
import numpy as np
import pandas as pd
import cv2

from sklearn.metrics import confusion_matrix #classification_report
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization

import keras
from keras.utils import np_utils
from keras.utils.vis_utils import plot_model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.models import model_from_json
from keras.models import load_model
from keras.utils.generic_utils import CustomObjectScope

from keras.callbacks import EarlyStopping, TensorBoard

import matplotlib.pyplot as plt
%matplotlib inline  
# for jupyter notebook enviornment. 


from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix #classification_report
import itertools  # for confusion matrix plot

from PIL import Image as pil_image


#import tensorflow as tf
#from tensorflow.python.framework import ops
from keras import backend as K

#path = '/python/DDSA/fer/fer2013/'
class_label = ['angry', 'happy','neutral']

n_class = len(class_label)

path = '/python/'  # location
os.chdir(path)

img_size = 48  # fer data size. 48 x 48
target_size = 48 #197 # minimum data size for specific net such as Inception, VGG, ResNet ...

epochs = 30  # n of times of training using entire data
batch_size = 16


In [9]:
def plot_hist(hist):
    plt.figure(0)
    fig, loss_ax = plt.subplots()
    acc_ax = loss_ax.twinx()

    loss_ax.plot(hist.history['loss'], 'y', label='train loss')
    loss_ax.plot(hist.history['val_loss'], 'r', label='val loss')
    loss_ax.set_xlabel('epoch')
    loss_ax.set_ylabel('loss')
    loss_ax.legend(loc='upper left')

    acc_ax.plot(hist.history['acc'], 'b', label='train acc')
    acc_ax.plot(hist.history['val_acc'], 'g', label='val acc')
    acc_ax.set_ylabel('accuracy')
    acc_ax.legend(loc='lower left')

    plt.show()
    plt.savefig('loss_accuracy_plot')
    plt.close()

# Make and plot confusion matrix. To see the detailed imformation about TP, TN of each classes.    
def make_confusion_matrix(model, x, y, normalize = True):
    predicted = model.predict(x)

    pred_list = []; actual_list = []
    for i in predicted:
        pred_list.append(np.argmax(i))
    for i in y:
        actual_list.append(np.argmax(i))

    confusion_result = confusion_matrix(actual_list, pred_list)
    plot_confusion_matrix(confusion_result, classes = class_label, normalize = normalize, title = 'Confusion_matrix')
    return confusion_result

def plot_confusion_matrix(cm, classes,
                          normalize = False,
                          title = 'Confusion matrix',
                          cmap = plt.cm.Blues):  
    if normalize:
        cm = cm.astype('float') / cm.sum(axis = 1)[:, np.newaxis]
        print("normalized")
    else:
        print('without normalization')

    print(cm)
    plt.figure(1)
    plt.imshow(cm, interpolation='nearest', cmap = cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment = "center",
                 color = "white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.savefig('Confusion_matrix')
    plt.close()
def data_arrange(x_data,y_data):
    # data class re-arrange
    
    # Angry vs neutral vs happy case
    x_angry = x_data[y_data==0]
    x_happy = x_data[y_data==3]
    x_neutral = x_data[y_data==6]
    
    y_angry = y_data[y_data==0]
    y_happy = y_data[y_data==3]
    y_neutral = y_data[y_data==6]
    
    # number of happy samples are twice  
    # To avoid class distribution bias. use only 50% sample of happy class
    x_happy_use, x_no, y_happy_use, y_no = train_test_split(x_happy, y_happy, test_size = 0.5, shuffle = True, random_state=33)
    
    #print('Before normalize:{a}\n'.format(a= x_angry[0]))
    xx = np.concatenate((x_angry, x_happy_use, x_neutral),axis=0)/255.0 #concatenate & normalized
    yy = np.concatenate((y_angry, y_happy_use, y_neutral), axis=0)
    yy[yy==3]=1
    yy[yy==6]=2
    
    xx = xx.reshape(-1, img_size,img_size)
    xx = np.stack((xx,)*3, -1 )  # to make fake RGB channel
    yy = np_utils.to_categorical(yy, n_class)
    print('After normalize x:{a} y:{b}\n'.format(a= xx.shape, b=yy.shape))     

    return xx, yy

def sample_plot(x_test, y_test):
    x_angry = x_test[y_test==0]
    a_f = np.squeeze(x_angry[1])
    plt.imshow(a_f, cmap='gray')
    

In [13]:
# data load
x_data = np.load('./x_data.npy')
y_data = np.load('./y_data.npy')

# 2. arrange the data. shape change, use specific class only, ...
x_data, y_data = data_arrange(x_data, y_data)

# 3. train / test split
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2, shuffle = True, random_state=33)
#np.shape(x_train)


After normalize x:(18589, 48, 48, 3) y:(18589, 3)



In [7]:
# mobile net

model_path = 'transfer_model_20'
with CustomObjectScope({'relu6': keras.layers.ReLU(6.),'DepthwiseConv2D': keras.layers.DepthwiseConv2D}):
        loaded_model = load_model(model_path+'.h5')    
        

In [14]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 50, 50, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 24, 24, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 24, 24, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 24, 24, 32)        0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 26, 26, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 24, 24, 32)        288       
__________

In [ ]:
loaded_model.compile(loss = categorical_crossentropy,
                      optimizer = Adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-7),
                      metrics = ['accuracy'])

early_stopping = EarlyStopping(monitor = 'val_acc', min_delta = 0.001, patience = 20, 
                                       verbose = 1, mode = 'max')
    

In [15]:
confusion_result = make_confusion_matrix(loaded_model, x_test, y_test, normalize = True) 

normalized
[[0.8019954  0.02225633 0.17574827]
 [0.13027853 0.71877808 0.1509434 ]
 [0.20199693 0.03763441 0.76036866]]


In [57]:
data_path = '/0GoogleDrive/tmp/cam_face/'
train_datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        zca_epsilon=1e-06,  # epsilon for ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        # randomly shift images horizontally (fraction of total width)
        width_shift_range=0.1,
        # randomly shift images vertically (fraction of total height)
        height_shift_range=0.1,
        shear_range=0.,  # set range for random shear
        zoom_range=0.,  # set range for random zoom
        channel_shift_range=0.,  # set range for random channel shifts
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        cval=0.,  # value used for fill_mode = "constant"
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False,  # randomly flip images
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0)

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
        data_path+'train',
        target_size=(48, 48),
        batch_size=16, color_mode = 'grayscale',
        class_mode='categorical', save_to_dir = './',save_prefix='aug')

validation_generator = test_datagen.flow_from_directory(
        data_path+'validation',
        target_size=(48, 48),
        batch_size=16,
        class_mode='categorical')

early_stopping = EarlyStopping(monitor = 'val_acc', min_delta = 0.001, patience = 20, 
                                       verbose = 1, mode = 'max')

Found 156 images belonging to 3 classes.
Found 39 images belonging to 2 classes.


In [30]:
batch_size=16
hist = loaded_model.fit_generator(
        train_generator,
        steps_per_epoch=30,
        epochs=10,
        validation_data=validation_generator,
        validation_steps=10,callbacks = [early_stopping])

Epoch 1/10
30/30 [==============================] - 27s 888ms/step - loss: 0.0240 - acc: 0.9937 - val_loss: 0.1335 - val_acc: 0.9310
Epoch 2/10
30/30 [==============================] - 27s 901ms/step - loss: 0.0314 - acc: 0.9903 - val_loss: 0.1632 - val_acc: 0.9310
Epoch 3/10
30/30 [==============================] - 26s 865ms/step - loss: 0.0103 - acc: 0.9958 - val_loss: 0.0183 - val_acc: 1.0000
Epoch 4/10
30/30 [==============================] - 24s 794ms/step - loss: 0.0219 - acc: 0.9958 - val_loss: 0.0702 - val_acc: 0.9828
Epoch 5/10
30/30 [==============================] - 26s 851ms/step - loss: 0.0314 - acc: 0.9937 - val_loss: 0.1432 - val_acc: 0.9483
Epoch 6/10
30/30 [==============================] - 27s 914ms/step - loss: 0.0075 - acc: 0.9979 - val_loss: 0.1323 - val_acc: 0.9483
Epoch 7/10
30/30 [==============================] - 26s 881ms/step - loss: 0.0128 - acc: 0.9979 - val_loss: 0.3089 - val_acc: 0.9310
Epoch 8/10
30/30 [==============================] - 28s 944ms/step - 

In [33]:
loaded_model.save_weights('./cam_model_weight.h5')


In [37]:
loaded_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 48, 48, 3)         0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 50, 50, 3)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 24, 24, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 24, 24, 32)        128       
_________________________________________________________________
conv1_relu (Activation)      (None, 24, 24, 32)        0         
_________________________________________________________________
conv_pad_1 (ZeroPadding2D)   (None, 26, 26, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 24, 24, 32)        288       
__________

In [38]:
model_name = 'mobile_transfer'
plot_model(loaded_model, to_file = model_name + '_net.png', show_shapes=True, show_layer_names=True)

In [36]:
confusion_result = make_confusion_matrix(loaded_model, x_test, y_test, normalize = False) 

without normalization
[[1303    0    0]
 [1113    0    0]
 [1302    0    0]]


In [53]:
loaded_model.layers[-3].output
str(loaded_model.get_layer(index=-3))[19:30]
#loaded_model.get_layer

'.Activation '

In [55]:
str(loaded_model.get_layer(index=-3)

In [59]:
1e-10/10

1.0000000000000001e-11

In [ ]:

### model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(n_class, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)




# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
